# Защита персональных данных клиентов  
Необходимо предложить механизм преобразования данных пользователей, который бы помог защитить личную информацию, но не отразился бы на предсказании модели линейной регрессии

In [51]:
import pandas as pd
import numpy as np
import numpy.linalg

from sklearn.linear_model import LinearRegression

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.metrics import r2_score

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.float_format', '{:.2f}'.format)

[1. Загрузка данных, обзор](#data_download)  
[2. Предобработка данных](#data_preprocessing)   
[3. Предложение метода защиты данных](#data_safety_method)    
[4. Доказательство корректности метода](#proof)  
[5. Прогрммирования линейной регрессии](#linear_programm)  
[6. Обучение и проверка собсвенной линейной регрессии до преобразования](#linear_before)  
[7. Обучение и проверка собственной линейной регресси после преобразования](#linear_after)  
[8. Обучение и проверка качества линейной регрессии до применения метода](#before)    
[9. Обучение и проверка качества линейной регрессии после применения метода](#after)    
[10. Выводы](#conclusion)  

<a id='data_download'></a>
### 1. Загрузка данных, обзор

In [28]:
try:
    data = pd.read_csv('/Users/ulia/Downloads/insurance.csv')
except FileNotFoundError:
    data = pd.read_csv('/datasets/insurance.csv')

In [29]:
display(data.head())
print()
display(data.info())

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.00,49600.00,1,0
1,0,46.00,38000.00,1,1
2,0,29.00,21000.00,0,0
3,0,21.00,41700.00,2,0
4,1,28.00,26100.00,0,0



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


None

<a id='data_preprocessing'></a>
### 2. Предобработка данных

In [30]:
data.duplicated().sum()

153

Удалим дубликаты:

In [31]:
data = data.drop_duplicates()
data.duplicated().sum()

0

Изменим названия столбцов:

In [32]:
data.columns = ['sex','age','salary','family_members','insurance_payments']
data.head()

,sex,age,salary,family_members,insurance_payments
0,1,41.00,49600.00,1,0
1,0,46.00,38000.00,1,1
2,0,29.00,21000.00,0,0
3,0,21.00,41700.00,2,0
4,1,28.00,26100.00,0,0


Переведем столбец "age" в числовой тип:  

In [33]:
data.age = data.age.astype('int')

In [34]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4847 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   sex                 4847 non-null   int64  
 1   age                 4847 non-null   int64  
 2   salary              4847 non-null   float64
 3   family_members      4847 non-null   int64  
 4   insurance_payments  4847 non-null   int64  
dtypes: float64(1), int64(4)
memory usage: 227.2 KB


Пердобработка завершена

<a id='data_safety_method'></a>
### 3. Предложение метода защиты данных

Умножим матрицу признаков **data[['sex', 'age', 'family_members', 'salary' ]]** на обратимую матрицу матрицу размером (4*4)  
Докажем, что качество предсказания модели линейной регрессии от данного  преобразования не изменится

 <a id='proof'></a>
### 4. Доказательство корректности метода

Необходимые формулы:  
  
$ (AB)^{T} = B^{T}A^{T} $  
$ (AB)^{-1} = B^{-1}A^{-1} $  
$ AA^{-1} = E = A^{-1}A $  

Чтобы матрица была обратимой, она должна быть квадратной.


Пусть $ X^{'} $ - матрица признаков $ m*n $  

Пусть $ W_{1} $ - вектор весов  $(1*m)$ , $W_{0}$  - скаляр

Тогда предсказание линейной регрессии можно записать:
$$Y = X^{'} *W_{1} + W_{0} $$
для упрощения расчетов добавим к матрице $ X^{'} $ единичный столбец, а скаляр $ W_{0}$ добавим в вектор $ W_{1} $

$$\begin{bmatrix} 1 & x_{12} & x_{13} & ... & x_{1n} \\ 1 & x_{21} & x_{22} & ... & x_{2n} \\ ...\\ 1 & x_{m2} & x_{m3} & ... & x_{mn}\end{bmatrix} *  \begin{bmatrix} w_{0} \\ w_{1} \\ ... \\ w_{n}\end{bmatrix}$$ 

Обозначим новую матрицу признаков с еденичным столбцом за $ X $ , а вектор весов - $ W $

**Формула предсказания линейной регрессии:**
$ Y = X*W $ 


Если принять метрику качества - минимальное значение **MSE** -  среднеквадратичноей ошибки, то для поиска минимального решения необходимо минимизировать:  

$ (Y - XW)^{T}(Y-XW) -> min $  


оптимальное решение относительно W находится приравниваем к нулю дифференциала этой функции по W   
 
В результате преобразований получим  

$ W = (X^{T}X)^{-1}X^{T}Y $

### Докажем, что умножение матрицы X на обратимую матрицу не ухудшит качество предсказаний

### Умножим матрицу признаков $ X $ на обратимую матрицу $ Q $ размерности (n*n) и найдем оптимальные веса $ W^{'} $

$ W^{'} = ((XQ)^{T}XQ)^{-1}(XQ)^{T}Y $  

$ W^{'} = (Q^{T}X^{T}XQ)^{-1}Q^{T}X^{T}Y $

$ W^{'} = Q^{-1}(X^{T}X)^{-1}(Q^{T})^{-1}Q^{T}X^{T}Y $

$ (Q^{T})^{-1}Q^{T}  = E $  
 
отсюда:

$ W^{'} = Q^{-1}(X^{T}X)^{-1}X^{T}Y $ , где  
$ (X^{T}X)^{-1}X^{T}Y $ равно $ W $

Получаем,что   

$ W^{'} = Q^{-1}*W $

Подставим $ Q^{-1}W $ вместо веса $ W^{'} $  в предсказание модели $ Y^{'} = XQW^{'} $ 

$ Y^{'} = XQ*Q^{-1}W $  
$ Y^{'} = XEW $  
$ Y^{'} = XW = Y $  

Доказали, что от умножения на обратимую матрицу, предсказания модели **не меняются**. Меняются только веса, те модель необходимо переобучить.

<a id='linear_programm'></a>
### 5.  Прогрммирование линейной регрессии

In [35]:
features =  data.drop('insurance_payments', axis=1)
target = data['insurance_payments']

In [36]:
class LinearRegression_custom:
    def fit(self, features_train, target_train):
        X = np.concatenate((np.ones((features_train.shape[0],1)), features_train), axis=1)
        Y = target_train
        W = np.linalg.inv(X.T.dot(X)) @ X.T @ Y
        self.W = W[1:]
        self.W0 = W[0]
    
    def predict(self, features_test):
        return features_test.dot(self.W) + self.W0


<a id=linear_before></a>
### 6. Обучение и проверка собсвенной линейной регрессии до преобразования

In [37]:
scaler = StandardScaler()
scaler.fit(features)
features_scaled = scaler.transform(features)

In [38]:

model = LinearRegression_custom()
model.fit(features_scaled, target)
predict_before_custom = model.predict(features_scaled)
print(round(r2_score(target, predict_before_custom),2))

0.43


<a id=linear_after></a>
### 7. Обучение и проверка собственной линейной регресси после преобразования 

Сгенерируем матрицу (4*4)

In [39]:
matrix = np.random.randint(1000, size = (4,4))

In [40]:
matrix.shape

(4, 4)

Проверим, что матрица обратима:

In [41]:
numpy.linalg.inv(matrix)

array([[ 0.00230375, -0.00062729,  0.00175238, -0.0021772 ],
       [-0.00211805,  0.00200987, -0.00066229,  0.00096407],
       [-0.00290442,  0.00172806,  0.00317764, -0.00155445],
       [ 0.00303454, -0.00218713, -0.00277009,  0.00242452]])

Матрица обратима  

Умножим матрицу признаков на  обратимую матрицу:

In [42]:
features_new_custom = features @ matrix

In [43]:
scaler.fit(features_new_custom)
features_scaled_new_custom = scaler.transform(features_new_custom)

In [44]:
model = LinearRegression_custom()
model.fit(features_scaled_new_custom, target)
predict_after_custom = model.predict(features_scaled_new_custom)
print(round(r2_score(target, predict_after_custom),2))

0.43


Видим, что метрика r2 не изменилась после преобразования. Каство предсказания модели не поменялось.

<a id='before'></a>
### 8. Обучение и проверка качества линейной регрессии до применения метода

In [45]:
features_train, features_test, target_train, target_test = train_test_split(features, target, random_state=123,test_size=0.4) 

In [46]:
reg = LinearRegression()
steps = [('scaler', StandardScaler()),('regression',reg)]
pipeline = Pipeline(steps=steps)
pipeline.fit(features_train, target_train)
predictions_before = pipeline.predict(features_test)
R_2 = r2_score(target_test, predictions_before)
round(R_2,2)

0.42

<a id='after'></a>
### 9. Обучение и проверка качества линейной регрессии после применения метода

Умножим матрицу признаков на  обратимую матрицу:

In [47]:
features_new = features @ matrix

In [48]:
features_new_train, features_new_test, target_new_train, target_new_test = train_test_split(features_new, target, random_state=123,test_size=0.4) 

In [49]:
reg = LinearRegression()
steps = [('scaler', StandardScaler()),('regression',reg)]
pipeline = Pipeline(steps=steps)
pipeline.fit(features_new_train, target_new_train)
predictions_after = pipeline.predict(features_new_test)
R_2_new = r2_score(target_new_test, predictions_after)
round(R_2_new,2)

0.42

In [50]:
print('Метрика R2 до преобразования равна {:.2f},после преобразовния {:.2f}'.format(R_2, R_2_new))

Метрика R2 до преобразования равна 0.42,после преобразовния 0.42


### Умножение на обратимую матрицу не привело к изменению качества работы модели

<a id='conclusion'></a>
### 10. Выводы


1. Для защиты данных клиентов предложили механизм сохранения данных клиентов - умножение матрицы признаков на обратимую матрицу
2. Проверили данный метод преобразования, доказали, что качество предсказания модели до и после уможения не меняется